In [1]:
import imageio
import numpy as np
import json
from matplotlib import pyplot as plt
from skimage.transform import warp
from scipy.ndimage import gaussian_filter
from scipy import ndimage
import math
import nibabel as nib
import utils
import random
import os

In [2]:
base_dir = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTENS'

width = 768
height = 768

n_x = 18
n_y = 18

sample_size = 4800
#sample_size = 10

start_sample = 10000
end_sample = start_sample + sample_size

p_dropout = 0.7
p_sinus = 0.7
p_rot = 0.7
p_scale = 0.7
p_shear = 0.7
p_global_trans = 0.7
p_local_trans = 0.7

# -----------
dropout_max = 0.25

x_sin_max = 0.15 * height
y_sin_max = 0.015 * width

rot_max = math.pi * 0.1125 # 22.5 deg

x_scale_max = 1.25
y_scale_max = 1.25

x_shear_max = math.pi * 0.05625 # 11.25 deg
y_shear_max = math.pi * 0.05625 # 11.25 deg

x_global_max = 0.2 * width
y_global_max = 0.2 * height

x_local_max = 0.005 * width
y_local_max = 0.005 * height

In [3]:
spacing_x = (width / 2.0) / n_x
spacing_y = (height / 2.0) / n_y

base_offset_x = width / 4.0
base_offset_y = height / 4.0

offset_x = spacing_x / 2.0 + base_offset_x
offset_y = spacing_y / 2.0 + base_offset_y

In [4]:
def random_decision(probability):
    return random.random() < probability

In [5]:
def apply_sinus(x, y, rand_1, rand_2): 
    y_ratio = (y - offset_y) / (spacing_y * (n_y - 1))
    y_sin = math.sin(y_ratio * math.pi) * y_sin_max * rand_1
    u = x - round(y_sin)
    
    x_ratio = (x - offset_x) / (spacing_x * (n_x - 1))
    x_sin = math.sin(x_ratio * math.pi) * x_sin_max * rand_2
    v = y - round(x_sin)
    
    return u,v

def apply_rot(x, y, rand_1):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    alpha = rot_max * rand_1
    
    vec = np.array([x, y])
    mat = np.array([[np.cos(alpha), -np.sin(alpha)],
                    [np.sin(alpha), np.cos(alpha)]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_scale(x, y, rand_1, rand_2):
    rand_1 = (rand_1 + 1.0) / 2.0
    rand_2 = (rand_1 + 1.0) / 2.0
    
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    x_scale = (1 / x_scale_max) + (x_scale_max - (1 / x_scale_max)) * rand_1
    y_scale = (1 / y_scale_max) + (y_scale_max - (1 / y_scale_max)) * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[x_scale, 0      ],
                    [0      , y_scale]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_shear(x, y, rand_1, rand_2):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    phi = x_shear_max * rand_1
    psi = y_shear_max * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[1          , np.tan(phi)],
                    [np.tan(psi), 1          ]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_global_trans(x, y, rand_1, rand_2):
    u = x + round(x_global_max * rand_1)
    v = y + round(y_global_max * rand_2)
    
    return u,v

def apply_local_trans(x, y):
    u = x + round(x_local_max * (random.random() * 2.0 - 1.0))
    v = y + round(y_local_max * (random.random() * 2.0 - 1.0))
    
    return u,v

def apply_dropout(rand):
    dropout = dropout_max * rand
    if random_decision(dropout):
        return True
        
    else:
        return False

In [6]:
def get_fixed_data(sample_index):
    is_dropout = random_decision(p_dropout)
    
    fixed = np.zeros((height, width))
    index_2_xy = dict()
    index = 0
    rand = random.random()
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            x = int(round(offset_x + (spacing_x * index_x)))
            
            drop = apply_dropout(rand) if (is_dropout and sample_index!=0) else False
            
            if not drop:
                fixed[y][x] = 1
                index_2_xy[index] = (x, y)
            
            index += 1
            
    return fixed, index_2_xy

In [7]:
def get_moving_data(rand_1, rand_2, sample_index, index_2_xy):
    is_sinus = random_decision(p_sinus)
    is_rot = random_decision(p_rot)
    is_scale = random_decision(p_scale)
    is_shear = random_decision(p_shear)
    is_global_trans = random_decision(p_global_trans)
    is_local_trans = random_decision(p_local_trans)
    
    moving = np.zeros((height, width))
    index_2_uv = dict()
    index = 0
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            if index in index_2_xy:
                x = int(round(offset_x + (spacing_x * index_x)))

                u = x
                v = y
                
                if sample_index != 0:
                    if is_sinus: u,v = apply_sinus(u, v, rand_1, rand_2)
                    if is_rot: u,v = apply_rot(u, v, rand_1) 
                    if is_scale: u,v = apply_scale(u, v, rand_1, rand_2) 
                    if is_shear: u,v = apply_shear(u, v, rand_1, rand_2) 
                    if is_global_trans: u,v = apply_global_trans(u, v, rand_1, rand_2) 
                    if is_local_trans: u,v = apply_local_trans(u, v) 
                
                if u < 0 or v < 0 or u > (width-1) or v > (height-1):
                    print('Additional attempt for sample index "{}"'.format(sample_index))
                    return None, None, False
                
                else:
                    moving[v][u] = 1
                    index_2_uv[index] = (u, v)

            index += 1
            
    return moving, index_2_uv, True

In [8]:
for sample_index in range(start_sample, end_sample):
    # ----- fixed -----
    fixed, index_2_xy = get_fixed_data(sample_index)
    
    if sample_index == 0:
        json_file = json.dumps(index_2_xy)
        f = open(base_dir + "/fix.json".format(sample_index), "w")
        f.write(json_file)
        f.close()

        fixed = utils.apply_smoothing(fixed, sigma=2, sigma_back=15)
        plt.imsave(base_dir + "/fix.png".format(sample_index), fixed, cmap="gray")

    # ----- moving -------
    is_valid = False
    
    while(not is_valid):
        rand_1 = random.random() * 2.0 - 1.0
        rand_2 = random.random() * 2.0 - 1.0
        
        moving, index_2_uv, is_valid = get_moving_data(rand_1, rand_2, sample_index, index_2_xy)
        
    json_file = json.dumps(index_2_uv)
    f = open(base_dir + "/raw/{}.json".format(sample_index), "w")
    f.write(json_file)
    f.close()

    moving = utils.apply_smoothing(moving, sigma=2, sigma_back=15)
    plt.imsave(base_dir + "/raw/{}_mov.png".format(sample_index), moving, cmap="gray")

Additional attempt for sample index "10004"
Additional attempt for sample index "10007"
Additional attempt for sample index "10028"
Additional attempt for sample index "10030"
Additional attempt for sample index "10047"
Additional attempt for sample index "10066"
Additional attempt for sample index "10072"
Additional attempt for sample index "10098"
Additional attempt for sample index "10109"
Additional attempt for sample index "10112"
Additional attempt for sample index "10112"
Additional attempt for sample index "10126"
Additional attempt for sample index "10133"
Additional attempt for sample index "10141"
Additional attempt for sample index "10147"
Additional attempt for sample index "10155"
Additional attempt for sample index "10160"
Additional attempt for sample index "10177"
Additional attempt for sample index "10181"
Additional attempt for sample index "10199"
Additional attempt for sample index "10200"
Additional attempt for sample index "10208"
Additional attempt for sample in

Additional attempt for sample index "12439"
Additional attempt for sample index "12472"
Additional attempt for sample index "12487"
Additional attempt for sample index "12514"
Additional attempt for sample index "12531"
Additional attempt for sample index "12534"
Additional attempt for sample index "12547"
Additional attempt for sample index "12549"
Additional attempt for sample index "12550"
Additional attempt for sample index "12551"
Additional attempt for sample index "12554"
Additional attempt for sample index "12567"
Additional attempt for sample index "12571"
Additional attempt for sample index "12574"
Additional attempt for sample index "12588"
Additional attempt for sample index "12591"
Additional attempt for sample index "12613"
Additional attempt for sample index "12620"
Additional attempt for sample index "12629"
Additional attempt for sample index "12651"
Additional attempt for sample index "12652"
Additional attempt for sample index "12654"
Additional attempt for sample in

Additional attempt for sample index "14573"
Additional attempt for sample index "14592"
Additional attempt for sample index "14607"
Additional attempt for sample index "14639"
Additional attempt for sample index "14651"
Additional attempt for sample index "14679"
Additional attempt for sample index "14689"
Additional attempt for sample index "14746"
Additional attempt for sample index "14759"
Additional attempt for sample index "14762"
Additional attempt for sample index "14795"


In [15]:
# Image start_sample will always be in the training
shuffled_list = list(range(start_sample+1, end_sample))
random.shuffle(shuffled_list)
shuffled_list.insert(0, start_sample)

if sample_size*0.15 % 1 > 0.0:
    raise AssertionError("Sample needs different split")

train_part = int(sample_size * 0.7)
val_part = int(sample_size * 0.15)
test_part = int(sample_size * 0.15)

train = shuffled_list[0:train_part] # 70 percent training
validation = shuffled_list[train_part:train_part+val_part] # 15 percent training
test = shuffled_list[train_part+val_part:sample_size] # 15 percent training

In [16]:
#fixed_sou = os.path.join(base_dir, "raw", "0.png")
#fixed_json_sou = os.path.join(base_dir, "raw", "0.json")

#fixed_des = os.path.join(base_dir, "train", "0.png")
#fixed_json_des = os.path.join(base_dir, "train", "0.json")

#os.rename(fixed_sou, fixed_des)
#os.rename(fixed_json_sou, fixed_json_des)

for index in train:
    moving_sou = os.path.join(base_dir, "raw", "{}_mov.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}.json".format(index))
    
    moving_des = os.path.join(base_dir, "train", "{}_mov.png".format(index))
    moving_json_des = os.path.join(base_dir, "train", "{}.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)

In [13]:
train.index(0)

ValueError: 0 is not in list

In [12]:
a.index(1)

0

In [17]:
for index in validation:
    moving_sou = os.path.join(base_dir, "raw", "{}_mov.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}.json".format(index))
    
    moving_des = os.path.join(base_dir, "validation", "{}_mov.png".format(index))
    moving_json_des = os.path.join(base_dir, "validation", "{}.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)

In [18]:
for index in test:
    moving_sou = os.path.join(base_dir, "raw", "{}_mov.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}.json".format(index))
    
    moving_des = os.path.join(base_dir, "test", "{}_mov.png".format(index))
    moving_json_des = os.path.join(base_dir, "test", "{}.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)